# WALLABY Database Notebook

Access the WALLABY database with Python commands. This uses the [Django models](https://docs.djangoproject.com/en/3.1/topics/db/models/) to create an mapping between our Python objects and the PostgreSQL database.

To access this notebook with Django shell locally

```
python3 manage.py shell_plus --notebook
```

In this notebook we will show you how to interact with the database with Python objects. We will:

1. Create mock detections
2. Add comments to detections
3. Add tags to detections
4. Use custom logic to define the sources catalogue

In [2]:
import os
import json
from datetime import datetime
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
# Import models

from run.models import Run
from instance.models import Instance
from detection.models import Detection

from sources.models import Sources
from comments.models import Comments
from tag.models import Tag

In [4]:
# Utils

import random
import string
from decimal import Decimal

def random_name():
    N = 5
    return ''.join(random.choice(string.ascii_letters) for x in range(N))

## 1. Create mock detections

In [5]:
# Create a mock run
query = Run.objects.filter(id=1)
if query.exists():
    run = query.first()
else:
    run = Run.objects.create(name="Test", sanity_thresholds=json.dumps("{}"))
                                                                       
# Create a mock instance
query = Instance.objects.filter(id=1)
if query.exists():
    instance = query.first()
else:
    instance = Instance.objects.create(
        run_id=run.id,
        filename="test",
        boundary={0,0,0,0},
        run_date=datetime.now(),
        flag_log=bytes(0),
        reliability_plot=bytes(0),
        log=bytes(0),
        parameters=json.dumps("{}"),
        version='0.0.0',
        return_code=0,
        stdout=bytes(0),
        stderr=bytes(0),
    )

In [6]:
# Create a bunch of random detections
for _ in range(20):
    if Detection.objects.count() < 20:
        Detection.objects.create(
            instance_id=instance.id,
            run_id=run.id,
            name=random_name(),
            access_url="https://test",
            access_format="txt",
            x=random.random(),
            y=random.random(),
            z=random.random(),
            x_min=0.0,
            x_max=1.0,
            y_min=0.0,
            y_max=1.0,
            z_min=0.0,
            z_max=1.0,
            n_pix=500.0,
            f_min=0.0,
            f_max=10.0,
            f_sum=20.0,
            rms=0.0,
            w20=0.0,
            w50=0.0,
            ell_maj=1.0,
            ell_min=0.0,
            ell_pa=0.0,
            ell3s_maj=1.0,
            ell3s_min=0.0,
            ell3s_pa=0.0,
            err_x=999.0,
            err_y=999.0,
            err_z=999.0,
            err_f_sum=999.0,
            unresolved=random.choice([True, False])
        )

## 2. Add comments 

In [7]:
# Add comment to that detection
Comments.objects.create(
    comment="I have selected this at random",
    detection=Detection.objects.get(id=5),
    added_at=datetime.now(),
    updated_at=datetime.now()
)

/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Comments.added_at received a naive datetime (2021-02-04 09:34:32.805570) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Comments.updated_at received a naive datetime (2021-02-04 09:34:32.805579) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


<Comments: Comments object (4)>

In [12]:
# Add another comment to another detection
Comments.objects.create(
    comment="This detection was really bad because of noise.",
    detection=Detection.objects.get(id=17),
    added_at=datetime.now(),
    updated_at=datetime.now()
)

/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Comments.added_at received a naive datetime (2021-02-04 09:36:32.963365) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Comments.updated_at received a naive datetime (2021-02-04 09:36:32.963373) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


<Comments: Comments object (5)>

In [14]:
# Select all comments
for c in list(Comments.objects.all()):
    print(c.detection.id, c.comment)

5 I have selected this at random
17 This detection was really bad because of noise.


In [18]:
# Select detections with comments

[Detection.objects.get(id=c.detection_id) for c in list(Comments.objects.all())]

[<Detection: Detection object (5)>, <Detection: Detection object (17)>]

## 3. Tagging

In [21]:
# Create some tags

Tag.objects.create(
    tag_name="Noise",
    description="Don't trust this as a source since the image was noisy",
    added_at=datetime.now()
)

Tag.objects.create(
    tag_name="Kinematics",
    description="Image is resolved and good for kinematics analysis",
    added_at=datetime.now()
)

Tag.objects.create(
    tag_name="Further Inspection",
    description="This image shows interesting features and should be inspected further",
    added_at=datetime.now()
)

/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Tag.added_at received a naive datetime (2021-02-05 01:46:39.003561) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Tag.added_at received a naive datetime (2021-02-05 01:46:39.013321) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/usr/local/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1367: RuntimeWarning: DateTimeField Tag.added_at received a naive datetime (2021-02-05 01:46:39.014852) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


<Tag: Tag object (3)>

In [25]:
# Tag the detections

TagDetection.objects.create(
    tag=Tag.objects.get(id=1),
    detection=Detection.objects.get(id=2)
)

TagDetection.objects.create(
    tag=Tag.objects.get(id=1),
    detection=Detection.objects.get(id=3)
)

TagDetection.objects.create(
    tag=Tag.objects.get(id=1),
    detection=Detection.objects.get(id=4)
)

TagDetection.objects.create(
    tag=Tag.objects.get(id=2),
    detection=Detection.objects.get(id=12)
)

TagDetection.objects.create(
    tag=Tag.objects.get(id=3),
    detection=Detection.objects.get(id=9)
)

TagDetection.objects.create(
    tag=Tag.objects.get(id=3),
    detection=Detection.objects.get(id=16)
)

<TagDetection: TagDetection object (10)>

In [34]:
# Retrieve detections with Tag 1.

[td.detection for td in TagDetection.objects.filter(tag__id=1)]

[<Detection: Detection object (2)>,
 <Detection: Detection object (2)>,
 <Detection: Detection object (3)>,
 <Detection: Detection object (2)>,
 <Detection: Detection object (3)>,
 <Detection: Detection object (4)>]

## 4. Select sources catalogue with custom logic

Suppose we want to select the final sources catalogue from the detections with the following rules:

* Anything tagged with "Noise" or "Further Inspection" cannot be included in this catalogue
* Detection x value < 0.7 and y value < 0.7 (Detections were randomly selected with x, y, z from 0 to 1)

We could do this with the ORM as such:

In [61]:
# Detections matching these criteria
catalogue = Detection.objects\
    .filter(x__lte=0.7, y__lte=0.7)\
    .exclude(id__in=[td.detection.id for td in TagDetection.objects.filter(tag__id=1) | TagDetection.objects.filter(tag__id=3)])

In [63]:
# Create source catalogue
[Sources.objects.create(detection=d) for d in catalogue]

[<Sources: Sources object (1)>,
 <Sources: Sources object (2)>,
 <Sources: Sources object (3)>,
 <Sources: Sources object (4)>,
 <Sources: Sources object (5)>,
 <Sources: Sources object (6)>,
 <Sources: Sources object (7)>,
 <Sources: Sources object (8)>]

# TODOs

* Provide science teams with access to this system
* Get some initial feedback for this approach